In [ ]:
cd("path to data folder")

In [ ]:
function strInt(x)
   return parse(Int, String(x))
end

function floatInt(x)
    return Int(tryparse(Float64, String(x)))
end

function strFloat(x)
    return  tryparse(Float64, String(x))
end

# Reading Data from txt file and creating Arrays, Vectors (S, up, low, etc.)
open("E_Coli/modelround5.txt") do f
    lines = readlines(f)
    global R = strInt(split(lines[2], ",")[2])
    global C = strInt(split(lines[2], ",")[3])
    global S = zeros(R, C)
    for ln in enumerate(lines[4:end])
        val = split(ln[2], ",")
        try
            i = floatInt(val[1])
            j = floatInt(val[2])
            v = strFloat(val[3])
            S[i, j] = v
        catch MethodError
            global upIndex = ln[1]+3
            break
        end
    end
    
    global fd_c = strInt(split(lines[upIndex+1], ",")[3])
    global U = zeros(C, fd_c)
    global L = zeros(C, fd_c)
    
    for ln in enumerate(lines[upIndex+3:end])
        val = split(ln[2], ",")
        try
            i = floatInt(val[1])
            j = floatInt(val[2])
            v = strFloat(val[3])
            U[i, j] = v
        catch e
            if isa(e, MethodError)
                global lowIndex = ln[1]+upIndex+2
                break
            else
                println(string("UP: ", e))
            end
        end
    end
    for ln in enumerate(lines[lowIndex+3:end])
        val = split(ln[2], ",")
        try
            i = floatInt(val[1])
            j = floatInt(val[2])
            v = strFloat(val[3])
            L[i, j] = v
        catch e
            if isa(e, MethodError)
                global kIndex = ln[1]+lowIndex+2
                break
            else
                println("LOW: ", e)
            end
        end
    end
    global K = strInt(split(lines[kIndex+3], ",")[2])
end

In [ ]:
using Convex, Hypatia

# creating variable for flux distribution matrix
fd = Variable(C, fd_c)

# expression for objective (sum of 2-norm)
expr = sum([norm(fd[j, :], 2) for j in 1:C])

# creating problem
prob = minimize(expr)

# creating 0-norm using function that was mentioned in PDF file
l0_norm = sum([exp(-square(norm(transpose(S * fd)[k,:], 2)) / (2 * (eps(1e-130)^2))) for k in 1:fd_c])

# adding transpose(SV) <= K constraint
prob.constraints += l0_norm <= K

# adding bounds for matrix fd(v) as constraints
for j in 1:C
    for k in 1:fd_c
        prob.constraints += fd[j,k] <= U[j, k]
        prob.constraints += fd[j,k] >= L[j, k]
    end
end

# solving problem using Hypatia solver
solve!(prob, () -> Hypatia.Optimizer(verbose=false))

# calculating SV from solved fd
sv = S * fd.value
prob.status

In [ ]:
tol = 2e-5

# checking output and printing values
ct1 = 0
ct2 = 0
for i in sv
    if 0 - tol <= i <= 0 + tol
        ct1 += 1
    else
        ct2 += 1
    end
end
println(string("SV zeros = ", ct1, " not zeros = ", ct2))
v_zeros = ct1
ct1 = 0
ct2 = 0
for i in fd.value
    if 0 - tol <= i <= 0 + tol
        ct1 += 1
    else
        ct2 += 1
    end
end
println(string("FD zeros = ", ct1, " not zeros = ", ct2))
sv_zeros = ct1
for j in 1:C
    for k in 1:fd_c
        if !(L[j, k] - tol <= fd.value[j, k] <= U[j, k] + tol)
            println(string("not in bounds = ",
                    L[j, k] - tol, " ",
                    fd.value[j, k], " ",
                    U[j, k] + tol, " | ",
                    j,",",k
                    ))
        end
    end
end
ct1 = 0
for j in 1:C
    temp = 0
    for k in 1:fd_c
        if !(0 - tol <= fd.value[j, k] <= 0 + tol)
            temp += 1
        end
    end
    if temp == 0
        ct1 += 1
    end
end
println(string("Zero Rows = ", ct1))           

In [ ]:
# writing ouput to file
open("../output.txt", "w") do f
    for j in 1:C
        for k in 1:fd_c
            if k == fd_c
                write(f, string(fd.value[j, k] + 0.0))
            else
                write(f, string(fd.value[j, k] + 0.0, ","))
            end
        end
        write(f, "\n")
    end
end